# TD learning+价值函数估计

## I. <font color=blue>价值函数估计法的出发点：</font>

- 到目前为止，前面的方法都将state or action value处理成table，如sarsa、Q-learning都是在估计table中每个位置的state-action对应的action value。<font color=norange>所以他们又称为tabular Sarsa和tabular Q-learning。</font>这些方法仅适用于state-action数量很少的场景。当state-action数量很大时，算法每次迭代只能处理table中单个元素，效率低。

- <font color=green>**一种改进思路**</font>：function approximation methods。指的是用函数代替table，通过求解函数中的参数来找到state-action与value之间的稳定关系。
$$\begin{align}
V^{\pi}(s) & = V_{\theta }^{\pi}(s) = f(s,\theta )\\
Q^{\pi}(s, a) & = Q_{\theta }^{\pi}(s, a) = h(s, a, \theta ) \\
\end{align}$$
  - f(·)和h(·)可以是各种mapping关系，包括神经网络。问题求解的目标不再是直接求V(s)和Q(s,a)，而转变成了求函数的参数$\theta$。
  - 提升算法效率的方式：参数的一次调整会影响所有state-action pair的价值，算法的优化效率更高
  - 代价是此时函数只是原价值函数的approximation，如果函数的表达力不够，那么不论参数如何优化都不能准确表达原价值函数。

- <font color=green>**方法调整**</font>：
  - sarsa和Q-learning等基于RM的TD算法求解的目标是价值函数Q(s, a)。而用function approximation之后，求解的目标变成给定函数求参数$\theta$。
  - 目标问题变化：和前面章节一样，先用state value function来说明思路。
$$\begin{align}
原问题：& V^{\pi}(S)\\
新问题：\\
& \underset{\theta}{min}J(\theta) 
 = \underset{\theta}{min}E\left ( V^{\pi}(S)-\hat{V}(S,\theta ) \right )^2  \\
\Leftrightarrow  \ \ &\nabla_{\theta }E\left ( V^{\pi}(S)-\hat{V}(S,\theta ) \right )^2  =0\\
\Leftrightarrow  \ \ &E\nabla_{\theta }\left ( V^{\pi}(S)-\hat{V}(S,\theta ) \right )^2=0\\
\Leftrightarrow  \ \ &E\left [\left ( V^{\pi}(S)-\hat{V}(S,\theta ) \right )\nabla_{\theta }\hat{V}(S,\theta )\right ]=0
\end{align}
$$
  - 仍然是期望形式，因此还是可以用RM算法迭代求解，对应RM的第二种典型应用：SGD形式
  - **由于没有$V^{\pi}$的真实值的样本，所以不能直接用RM算法，和上一章中TD learning的思路一样，改用当前时刻对$V^{\pi}$的‘最佳估计’。**

- <font color=red>**代价：TD learning加上function approximation之后，除非特定条件下，否则不再具备almost surely convergence的性质。**</font>

## II. 用function approximation做TD learning of $V^{\pi}(S)$

### II.1 基础方法：function approximation + SGD+MC

#### II.1.1 目标函数

$$
\begin{align}
\underset{w}{min}J(w) 
& = \underset{w}{min}E\left [\left ( v^{\pi}(S)-\hat{v}(S,w) \right )^2 \right ] \\
& = \underset{w}{min}\sum _{s\in \mathcal{S}}d^{\pi}(s)[v^{\pi}(s)-\hat{v}(s,w)]^2 \\
w & = \underset{w}{argmin}J(w) \\
  & = \underset{w}{argmin} E\left [\left ( v^{\pi}(S)-\hat{v}(S,w)\right )^2 \right ]
\end{align}
$$
- $v^{\pi}(S)$和$\hat{v}(S,w)$分别是给定策略$\pi$时，所有state value的真实值和估计值。
- 这里的期望是针对状态$s\in S$的分布而言。

#### II.1.2 求解优化问题

1. 等价目标函数:
$$J(w) = \frac{1}{2} E\left [\left ( v^{\pi}(S)-\hat{v}(S,w)\right )^2 \right ] $$
2. 梯度：
$$
\begin{align}
\nabla J(w) 
& = \frac{1}{2} \nabla_w E\left [\left ( v^{\pi}(S)-\hat{v}(S,w)\right )^2 \right ] \\
& = -E\left [v^{\pi}(S)-\hat{v}(S,w)\right  ]\nabla_w \hat{v}(S,w)
\end{align}
$$
3. SGD算法：
$$
\begin{align}
w_{t+1} &  = w_t - \alpha_t\nabla_w J(w_t) \\
&= w_t + \alpha_t\left [v^{\pi}(s_t)-\hat{v}(s_t,w_t)\right  ]\nabla_w \hat{v}(s_t,w_t)
\end{align}
$$
4. 问题：<font color=red>**由于$v^{\pi}(s_t)$未知，因此无法直接使用SGD。**</font>
5. 一种简单的解决方法：Monte Carol方法。用episode的信息来估计所有状态$s$的state value，然后代入SGD求解。
6. 引出新的问题，MC只能在抽出很多episodes之后才能计算，如果要实时处理样本，需要新的方法。

### II.2 function approximation + TD learning

- 方法：用一个函数或者DNN来learn状态价值函数。
  - 以参数为w的DNN为例，迭代方式为：
    1. 随机初始化$w_0$
    2. 迭代，直到达到指定的收敛精度：
       - 抽取一个time-step样本$(s_t, a_t, r_t)$
       - 按照下面迭代式更新网络参数$w$
    3. 从得到的价值函数提取策略

- 迭代式为：
$$\color{Blue} {w_{t+1} = w_t + \alpha_t\left [r_{t+1}+\gamma \hat{v}(s_{t+1,w_t})-\hat{v}(s_t,w_t)\right  ]\nabla_w \hat{v}(s_t,w_t)}$$

## III. 用function approximation做TD learning of $Q^{\pi}(S,A)$

### III.1 Sarsa with function approximation

#### III.1.1 迭代过程推导

- 目标问题：
$$\begin{align}
&\underset{\theta}{min}J(\theta) 
 = \underset{\theta}{min}E\left ( Q^{\pi}(S,A)-\hat{Q}(S,A,\theta ) \right )^2  \\
\Leftrightarrow  \ \ &\nabla_{\theta }E\left ( Q^{\pi}(S,A)-\hat{Q}(S,A,\theta ) \right )^2  =0\\
\Leftrightarrow  \ \ &E\nabla_{\theta }\left ( Q^{\pi}(S,A)-\hat{Q}(S,A,\theta ) \right )^2=0\\
\end{align}$$

- RM变量：
$$\begin{align}
g(\theta ) 
& = E\nabla_{\theta }\left ( Q^{\pi}(S,A)-\hat{Q}(S,A,\theta ) \right )^2\\
& = -E\left [\left ( Q^{\pi}(S,A)-\hat{Q}(S,A,\theta ) \right )\nabla_{\theta }\hat{Q}(S,A,\theta )\right ] \\
\tilde g(\theta_t ,\eta_t ) 
& = -\left[ Q^{\pi}(s_t,a_t)-\hat{Q}(s_t,a_t,\theta_t) \right ]\nabla_{\theta }\hat{Q}(s_t,a_t,\theta )|_{\theta =\theta_t}
\end{align}$$

- RM迭代式：
$$\begin{align}
\theta_{t+1} 
& = \theta_t -\alpha_t(\theta_t)\tilde g(\theta_t ,\eta_t ) \\
& = \theta_t +\alpha_t(\theta_t)\left [ Q^{\pi}(s_t,a_t)-\hat{Q}(s_t,a_t,\theta_t) \right ]\nabla_{\theta }\hat{Q}(s_t,a_t,\theta )|_{\theta =\theta_t}
\end{align}$$

- 由于$ Q^{\pi}(s_t,a_t)$未知，和tabular TD中一样，将其替换为当前时刻的最佳估计量得到新的迭代式：
$$\begin{align}
\theta_{t+1} 
& = \theta_t +\alpha_t(\theta_t)\left [ Q^{\pi}(s_t,a_t)-\hat{Q}(s_t,a_t,\theta_t) \right ]\nabla_{\theta }\hat{Q}(s_t,a_t,\theta )|_{\theta =\theta_t} \\
& \rightarrow  \theta_t +\alpha_t(\theta_t)\left [ r_{t+1}+\gamma Q^{\pi}(s_{t+1},a_{t+1})-\hat{Q}(s_t,a_t,\theta_t) \right ]\nabla_{\theta }\hat{Q}(s_t,a_t,\theta )|_{\theta =\theta_t} \\
& \rightarrow \theta_t +\alpha_t(\theta_t)\left [ r_{t+1}+\gamma \hat Q(s_{t+1},a_{t+1}, \theta _t)-\hat{Q}(s_t,a_t,\theta_t) \right ]\nabla_{\theta }\hat{Q}(s_t,a_t,\theta )|_{\theta =\theta_t} \\
\end{align}
$$

#### III.1.2 算法伪码

- <img src="./pics/sarsaapprox.png" alt="alt text" width="560"/>


### III.2 Q-learning with function approximation

- 迭代式:
$$\color{Blue} {w_{t+1} = w_t + \alpha_t\left [r_{t+1}+\gamma\underset{a\in \mathcal{A}}{max}\ \hat{q}(s_{t+1},a,w_t)-\hat{q}(s_t,a_t,w_t)\right  ]\nabla_w \hat{q}(s_t,a_t,w_t)}$$

## IV. Deep Q-learning

- 思想：function approximation使用deep neural network
- 迭代式：
$$\color{Blue} {w_{t+1} = w_t + \alpha_t\left [r_{t+1}+\gamma\underset{a\in \mathcal{A}}{max}\ \hat{q}(s_{t+1},a,w_t)-\hat{q}(s_t,a_t,w_t)\right  ]\nabla_w \hat{q}(s_t,a_t,w_t)}$$
  - 其中$q(s_t,a_t,w_t)$是DNN

- 算法伪码
  - <img src="./pics/deepqlearning.png" alt="alt text" width="560"/>


- 算法特征：
  1. off-policy：有两个网络，behavior network用来explore environment，获得训练数据。target network用于迭代求目标参数。target network中的参数每次迭代都在改变；隔一段时间behavior network中的参数更新为target network的参数，在此之前，behavior network中的参数不像target network中那样每次迭代都改变。
  2. replay buffer：存放behavior network获得的数据。每个{s, a, r, s'}构成一个样本点，behavior network得到的episodes都被拆散成单个的样本点存入replay buffer。每次迭代时都从buffer中取出一个minibatch的样本来做训练。抽样采用以(s,a)值为index的uniform sample。
     - <font color=green>这里必须拆开成单个样本点做uniform sample是因为：不拆开的话样本点之间的相关性太强，他们都是基于当前策略连续抽的episodes。拆开后做有放回抽样可以打破这里的相关性，同时当个样本可能多次被抽中也提高了data efficiency。</font>

### 1. limit distribution的充要性证明

- 充分性证明：
$$
\lim_{k \to \infty} d_k^T =d_0^T\lim_{k \to \infty} (P^{\pi })^k= d_0^T\mathbb{1}_n(d^{\pi})^T = (d^{\pi})^T
$$
- 必要性证明：
$$
\begin{align}
\because &\ \lim_{k \to \infty} d_k^T  = d_0^T\lim_{k \to \infty} (P^{\pi })^k， (d^{\pi})^T  = d_0^T\mathbb{1}_n(d^{\pi})^T 
\\
if\ & \lim_{k \to \infty} d_k^T  = (d^{\pi})^T\ ,\  then \ \  \ d_0^T\lim_{k \to \infty} (P^{\pi })^k  = d_0^T\mathbb{1}_n(d^{\pi})^T  \\
\Rightarrow & \ d_0^T \left [\lim_{k \to \infty} (P^{\pi })^k - \mathbb{1}_n(d^{\pi})^T\right ] =0 \\
\Rightarrow & \ \lim_{k \to \infty} (P^{\pi })^k - \mathbb{1}_n(d^{\pi})^T =0 
\end{align}
$$